# Data Ingestion from Amazon S3 to Databricks

## Data Sources
- MDM: Contains data from ebx mdm.


In [0]:
%python
from pyspark.sql.functions import current_timestamp, StringType, col, regexp_replace
import dlt
from pyspark.sql.types import LongType, IntegerType, DecimalType, StringType

In [0]:
%python

bucket_prefix = "s3://unfi-data-landing-dev-s3/dev/hvr/ebx/mdm/"

folders = dbutils.fs.ls(bucket_prefix)

exclude_folders = ["temp/","ebx_hg_item/","ebx_hg_customerorderingaccount/","ebx_hg_itemstaging00/"]
include_folders = [""]
# Filter out the folder to exclude (e.g., "ebx_cm_ord_acct")
table_to_bucket = {
    folder.name: folder.path
    for folder in folders
    if folder.name not in exclude_folders
}

#"ebx_hg_item/",

In [0]:
table_to_bucket

{'ebx_bp_individ_address/': 's3://unfi-data-landing-dev-s3/dev/hvr/ebx/mdm/ebx_bp_individ_address/',
 'ebx_bp_individ_contact/': 's3://unfi-data-landing-dev-s3/dev/hvr/ebx/mdm/ebx_bp_individ_contact/',
 'ebx_bp_individ_email_addr/': 's3://unfi-data-landing-dev-s3/dev/hvr/ebx/mdm/ebx_bp_individ_email_addr/',
 'ebx_bp_individ_phone_number/': 's3://unfi-data-landing-dev-s3/dev/hvr/ebx/mdm/ebx_bp_individ_phone_number/',
 'ebx_bp_individual/': 's3://unfi-data-landing-dev-s3/dev/hvr/ebx/mdm/ebx_bp_individual/',
 'ebx_bp_oranization_contact/': 's3://unfi-data-landing-dev-s3/dev/hvr/ebx/mdm/ebx_bp_oranization_contact/',
 'ebx_bp_org_address/': 's3://unfi-data-landing-dev-s3/dev/hvr/ebx/mdm/ebx_bp_org_address/',
 'ebx_bp_org_email_addr/': 's3://unfi-data-landing-dev-s3/dev/hvr/ebx/mdm/ebx_bp_org_email_addr/',
 'ebx_bp_org_phone_number/': 's3://unfi-data-landing-dev-s3/dev/hvr/ebx/mdm/ebx_bp_org_phone_number/',
 'ebx_bp_organization/': 's3://unfi-data-landing-dev-s3/dev/hvr/ebx/mdm/ebx_bp_organi

In [0]:
modified_columns = ["vendor_","item_","enterprisebusinesspartyid","id"]
for column in modified_columns:
    print(column)

vendor_
item_
enterprisebusinesspartyid
id


In [0]:
#print(table_to_bucket)

for table_name, suffix in table_to_bucket.items():
    bucket_path = suffix
    tab_name = table_name.replace("/","")
    #print(tab_name)
    @dlt.create_table(name=tab_name,
                comment=f"{tab_name} data",
                table_properties={
                    "quality": "bronze"
                })
    
    def date_python_table(bucket_path=bucket_path, tab_name=tab_name):
        df = (
            spark.readStream.format("cloudFiles") \
                .option("cloudFiles.format", "parquet") \
                .option("cloudFiles.inferColumnTypes", "true") \
                .option("inferSchema", "true") \
                .option("cloudFiles.schemaEvolutionMode", "addNewColumns") \
                .option("overwriteSchema", "true") \
                .load(bucket_path) 
        )
 
        df = df.select([col(column).cast("string") for column in df.columns])
        df = df.withColumn('created_on', current_timestamp()) 

        return df

Name,Type
t_last_user_id,string
t_creator_id,string
t_creation_date,string
t_last_write,string
vendortype,string
description,string
technical_rowinactivedate,string
hvr_change_op,string
hvr_change_sequence,string
hvr_change_time,string


In [0]:
print(bucket_prefix)

s3://unfi-data-landing-dev-s3/dev/hvr/ebx/mdm/
